In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
# 創建一個 Options 物件
options = Options()

options.add_argument("--start-maximized") # Chrome 瀏覽器在啟動時最大化視窗
options.add_argument("--incognito") # 無痕模式
options.add_argument("--disable-popup-blocking") # 停用 Chrome 的彈窗阻擋功能。
# options.add_argument('--headless')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

driver.get('https://today.line.me/tw/v2/movie/incinemas/playing')

time.sleep(3)  # 等待網頁載入完成
movie_list = driver.find_elements(By.CLASS_NAME, 'detailListItem.movieListing-movie')

last_movie = None
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 滾動到網頁底部
    time.sleep(3)  # 等待網頁載入完成
    movie_list = driver.find_elements(By.CLASS_NAME, 'detailListItem.movieListing-movie')
    if last_movie == movie_list[-1]:
        break
    last_movie = movie_list[-1]

    print(f"Movie count: {len(movie_list)}")

In [ ]:
star_grade_mapping = {
    "M12 18.344l-5.81 3.609c-.147.091-.34.046-.43-.1-.043-.07-.057-.152-.04-.23l1.469-6.96-5.09-4.736c-.126-.118-.133-.316-.016-.442.052-.056.122-.091.198-.099l6.746-.68 2.684-6.513c.066-.16.249-.235.408-.17.077.032.138.094.17.17l2.684 6.514 6.746.68c.172.017.297.17.28.342-.008.075-.043.146-.099.198l-5.09 4.736 1.47 6.96c.036.169-.072.334-.24.37-.08.017-.161.002-.23-.04L12 18.343z": 2,
    "M12.12 2.024c.076.031.137.093.169.17l2.684 6.513 6.746.68c.172.017.297.17.28.342-.008.075-.043.146-.099.198l-5.09 4.736 1.47 6.96c.036.169-.072.334-.24.37-.08.017-.161.002-.23-.04L12 18.343l-5.81 3.61c-.147.091-.34.046-.43-.1-.043-.07-.057-.152-.04-.23l1.469-6.96-5.09-4.736c-.126-.118-.133-.316-.016-.442.052-.056.122-.091.198-.099l6.746-.68 2.684-6.513c.066-.16.249-.235.408-.17zM12 6.463v9.651l3.662 2.275-.925-4.383 3.316-3.086-4.398-.443L12 6.463z": 1,
    "M12.12 2.024c.076.031.137.093.169.17l2.684 6.513 6.746.68c.172.017.297.17.28.342-.008.075-.043.146-.099.198l-5.09 4.736 1.47 6.96c.036.169-.072.334-.24.37-.08.017-.161.002-.23-.04L12 18.343l-5.81 3.61c-.147.091-.34.046-.43-.1-.043-.07-.057-.152-.04-.23l1.469-6.96-5.09-4.736c-.126-.118-.133-.316-.016-.442.052-.056.122-.091.198-.099l6.746-.68 2.684-6.513c.066-.16.249-.235.408-.17zm1.6 8.365L12 6.216l-1.72 4.173-4.549.458 3.43 3.191-.961 4.546 3.8-2.36 3.799 2.36-.96-4.546 3.429-3.191-4.548-.458z": 0,
}


def get_movie_comments(url):
    driver.execute_script("window.open('about:blank', '_blank');") # 開啟分頁
    new_tabs = driver.window_handles
    driver.switch_to.window(new_tabs[-1])
    driver.get(url)

    comment_bth = driver.find_element(By.XPATH, "//*[starts-with(text(),'網友短評')]")
    comment_bth.click()

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 滾動到網頁底部
    time.sleep(2)  # 等待網頁載入完成

    rating_comment_cards = driver.find_elements(By.CLASS_NAME, 'ratingCommentItemCard')

    all_comments = []

    for rating_comment_card in rating_comment_cards:
        comment = rating_comment_card.find_element(By.CLASS_NAME, 'ratingCommentItemContent')
        stars = rating_comment_card.find_elements(By.CSS_SELECTOR, '.ratingStars.css-4rdtjp .ratingStar')
        
        grade = 0
        for star in stars:
            star_path = star.find_element(By.TAG_NAME, 'path').get_attribute('d')
            grade += star_grade_mapping.get(star_path, 0)
            
        all_comments.append({'grade': grade, 'comment': comment.text})

    driver.close()
    driver.switch_to.window(new_tabs[0])
    return all_comments

In [ ]:
movie_info_and_comments = {}

for movie in movie_list:
    title = movie.find_element(By.CLASS_NAME, 'detailListItem-titles').text.replace('\n', ' (') + ')'
    average_star = movie.find_element(By.CLASS_NAME, 'iconInfo.detailListItem-iconInfo').text
    # 使用 CSS Selector > 選取子元素
    info = movie.find_element(By.CSS_SELECTOR, '.detailListItem-status > span.text').text
    # 使用 get_attribute 取得屬性值
    url = movie.find_element(By.CLASS_NAME, 'detailListItem-bookingButton').get_attribute('href')
    print(f"{title} | {average_star} | {info} | {url}")

    movie_info_and_comments[title] = {
        'average_star': average_star,
        'info': info,
        'url': url,
        'comments': []
    }

    movie_info_and_comments[title]['comments'] = get_movie_comments(url)

    break

In [ ]:
print(movie_info_and_comments)